In [ ]:
import xgboost as xgb
import pandas as pd
import time

In [ ]:
# DATASET 1
train_features = pd.read_csv('dataset1_train_features.csv')
test_features = pd.read_csv('dataset1_test_features.csv')

train_types = []

for row in train_features['Type']:
    if row == 'Class':
        train_types.append(1)
    else:
        train_types.append(0)
        
train_features['Type_encode'] = train_types

test_types = []

for row in test_features['Type']:
    if row == 'Class':
        test_types.append(1)
    else:
        test_types.append(0)
        
test_features['Type_encode'] = test_types

X_train = train_features.loc[:, 'Ngram1_Entity':'Type_encode']
y_train = train_features['Match']

X_test = test_features.loc[:, 'Ngram1_Entity':'Type_encode']
y_test = test_features['Match']

df_train = train_features.loc[:, 'Ngram1_Entity':'Type_encode']
df_train['Match'] = train_features['Match']

df_test = test_features.loc[:, 'Ngram1_Entity':'Type_encode']
df_test['Match'] = test_features['Match']

X_train = X_train.fillna(value=0)
X_test = X_test.fillna(value=0)

train = pd.read_csv('dataset1_train.csv')
test = pd.read_csv('dataset1_test.csv')

In [ ]:
N = 500000

dtrain = xgb.DMatrix(X_train.sample(n = N, replace = True), label=y_train.sample(n = N, replace = True))
dtest = xgb.DMatrix(X_test.sample(n = N, replace = True), label=y_test.sample(n = N, replace = True))

# min_child_weight 10, gamma 2.0, subsample 0.8, colsample_bytree 0.8
# max_depth 5
param = {'silent': 0,
         'objective': 'binary:logistic',
         'min_child_weight': 10,
         'gamma': 2.0,
         'subsample': 0.8,
         'colsample_bytree': 0.8,
         'max_depth': 5
        }

param['nthread'] = 4
param['eval_metric'] = 'logloss'

evallist = [(dtest, 'eval'), (dtrain, 'train')]

plst = param.items()

num_round = 100
start = time.time()
bst = xgb.train(plst, dtrain, num_round, evallist, verbose_eval=True)
end = time.time()
print('Training time', end - start)

start = time.time()
y_prob = bst.predict(dtest)
end = time.time()
print('Testing time', end - start)